# Notebook ejecución
## Este notebook contiene todos los pasos tomados para desarrollar el código final alojado en main.py
### El analisis de los datasets y las explicaciones pertinentes estarán como celdas md o comentarios en el código.

#### Importamos las librerias

In [972]:
from openpyxl import load_workbook
import pandas as pd
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.styles import Border, Side

### Abrimos el workbook donde iremos guardando los datos del resumen

In [973]:
data = load_workbook("data/Financial Sample.xlsx")
sheet = data.active
sheet.title = "Datos originales"
#Creamos la hoja donde guardaremos los resumenes.
data.create_sheet("Resumenes")
data.active = data["Resumenes"]
sheet = data.active

#### Importamos el dataset
En un primer vistazo al excel que vamos a usar como ejemplo de datos financieros (https://learn.microsoft.com/es-es/power-bi/create-reports/sample-financial-download) vemos que solo tiene una hoja, por lo que será fácil pasarlo a un objeto de pandas.

In [974]:
df = pd.read_excel("data/Financial Sample.xlsx", engine = "openpyxl")

#### Analizamos el dataset

In [975]:
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Channel Partners,United States of America,Carretera,High,386.0,3,12,4632.0,463.20,4168.80,1158.0,3010.80,2013-10-01,10,October,2013
1,Government,United States of America,Montana,High,2328.0,5,7,16296.0,1629.60,14666.40,11640.0,3026.40,2014-09-01,9,September,2014
2,Channel Partners,United States of America,Paseo,High,386.0,10,12,4632.0,463.20,4168.80,1158.0,3010.80,2013-10-01,10,October,2013
3,Enterprise,United States of America,Carretera,High,3445.5,3,125,430687.5,43068.75,387618.75,413460.0,-25841.25,2014-04-01,4,April,2014
4,Enterprise,France,Carretera,High,1482.0,3,125,185250.0,18525.00,166725.00,177840.0,-11115.00,2013-12-01,12,December,2013


In [976]:
df.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       ' Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

In [977]:
df.shape

(700, 16)

In [978]:
df.describe()

,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Year
count,700.000000,700.000000,700.000000,7.000000e+02,700.000000,7.000000e+02,700.000000,700.000000,700,700.000000,700.000000
mean,1608.294286,96.477143,118.428571,1.827594e+05,13150.354629,1.696091e+05,145475.211429,24133.860371,2014-04-28 21:36:00,7.900000,2013.750000
min,200.000000,3.000000,7.000000,1.799000e+03,0.000000,1.655080e+03,918.000000,-40617.500000,2013-09-01 00:00:00,1.000000,2013.000000
25%,905.000000,5.000000,12.000000,1.739175e+04,800.320000,1.592800e+04,7490.000000,2805.960000,2013-12-24 06:00:00,5.750000,2013.750000
50%,1542.500000,10.000000,20.000000,3.798000e+04,2585.250000,3.554020e+04,22506.250000,9242.200000,2014-05-16 12:00:00,9.000000,2014.000000
75%,2229.125000,250.000000,300.000000,2.790250e+05,15956.343750,2.610775e+05,245607.500000,22662.000000,2014-09-08 12:00:00,10.250000,2014.000000
max,4492.500000,260.000000,350.000000,1.207500e+06,149677.500000,1.159200e+06,950625.000000,262200.000000,2014-12-01 00:00:00,12.000000,2014.000000
std,867.427859,108.602612,136.775515,2.542623e+05,22962.928775,2.367263e+05,203865.506118,42760.626563,NaN,3.377321,0.433322


#### Revisamos los valores nulos

In [979]:
df.isnull().sum()

Segment                 0
Country                 0
Product                 0
Discount Band          53
Units Sold              0
Manufacturing Price     0
Sale Price              0
Gross Sales             0
Discounts               0
 Sales                  0
COGS                    0
Profit                  0
Date                    0
Month Number            0
Month Name              0
Year                    0
dtype: int64

#### Al ver que solo hay uno, la columna Discount Band, vemos de que se trata y si es importante quitar los nulos

In [980]:
df["Discount Band"].value_counts()

Discount Band
High      245
Medium    242
Low       160
Name: count, dtype: int64

Por ser un valor categórico donde el valor nulo tiene un significado (no hay descuento) se imputará un nuevo valor "No discount".

In [981]:
df["Discount Band"].fillna(value="No discount", inplace=True)

C:\Users\vicbr\AppData\Local\Temp\ipykernel_45652\3912698363.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Discount Band"].fillna(value="No discount", inplace=True)


In [982]:
df["Discount Band"].value_counts()

Discount Band
High           245
Medium         242
Low            160
No discount     53
Name: count, dtype: int64

### Analisis de features que vamos a utilizar para el dashboard

In [983]:
df.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       ' Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

In [984]:
df["Country"].value_counts()

Country
United States of America    140
France                      140
Mexico                      140
Germany                     140
Canada                      140
Name: count, dtype: int64

#### Vamos sacando insights de los datos de venta de cada país

In [985]:
ventas_brutas_por_pais = df.groupby("Country")["Gross Sales"].sum().round(0)
ventas_brutas_por_pais.name = "Ventas Brutas (€)"
ventas_brutas_por_pais

Country
Canada                      26932164.0
France                      26081674.0
Germany                     24921468.0
Mexico                      22726935.0
United States of America    27269358.0
Name: Ventas Brutas (€), dtype: float64

In [986]:
promedio_ventas = df.groupby("Country")["Gross Sales"].mean().round(0)
promedio_ventas.name = "Promedio Ventas (€)"
promedio_ventas

Country
Canada                      192373.0
France                      186298.0
Germany                     178010.0
Mexico                      162335.0
United States of America    194781.0
Name: Promedio Ventas (€), dtype: float64

In [987]:
total_ganancias = df.groupby("Country")["Profit"].sum().round(0)
total_ganancias.name = "Ganancias totales (€)"
total_ganancias

Country
Canada                      3529229.0
France                      3781021.0
Germany                     3680389.0
Mexico                      2907523.0
United States of America    2995541.0
Name: Ganancias totales (€), dtype: float64

In [988]:
df["Profit_margin"] = df["Profit"]/df["Gross Sales"]
margen_ganancia_medio_por_pais = df.groupby("Country")["Profit_margin"].mean().round(6)*100
margen_ganancia_medio_por_pais.name = "Margen de ganancia medio (%)"
margen_ganancia_medio_por_pais

Country
Canada                      25.7194
France                      26.2337
Germany                     26.5765
Mexico                      25.4622
United States of America    25.9765
Name: Margen de ganancia medio (%), dtype: float64

In [989]:
promedio_unidades_vendidas = df.groupby("Country")["Units Sold"].mean().round(2)
promedio_unidades_vendidas.name = "Promedio Unidades vendidas (ud)"
promedio_unidades_vendidas

Country
Canada                      1767.35
France                      1720.94
Germany                     1439.24
Mexico                      1452.32
United States of America    1661.62
Name: Promedio Unidades vendidas (ud), dtype: float64

In [990]:
df['Ingresos'] = df['Sale Price'] * df['Units Sold']
ingresos_totales_por_pais = df.groupby('Country')['Ingresos'].sum().round(0)
ingresos_totales_por_pais.name = "Ingresos totales (€)"
ingresos_totales_por_pais

Country
Canada                      26932164.0
France                      26081674.0
Germany                     24921468.0
Mexico                      22726935.0
United States of America    27269358.0
Name: Ingresos totales (€), dtype: float64

In [991]:
ganancias_totales_por_pais = df.groupby('Country')['Profit'].sum().round(0)
ganancias_totales_por_pais.name = "Ganancias totales (€)"
ganancias_totales_por_pais

Country
Canada                      3529229.0
France                      3781021.0
Germany                     3680389.0
Mexico                      2907523.0
United States of America    2995541.0
Name: Ganancias totales (€), dtype: float64

In [992]:
df['Year Growth'] = df.groupby('Country')['Gross Sales'].pct_change()
crecimiento_anual_por_pais = df.groupby('Country')['Year Growth'].mean().round(2)
crecimiento_anual_por_pais.name = "Crecimiento anual (%)"
crecimiento_anual_por_pais

Country
Canada                      4.95
France                      5.87
Germany                     5.43
Mexico                      7.75
United States of America    7.88
Name: Crecimiento anual (%), dtype: float64

#### Añadimos los insights a la hoja de resúmenes

In [993]:
paises = ["Estadísticas","Canada", "Francia", "Alemania", "Mexico", "USA"]
estadisticas = {ventas_brutas_por_pais.name : ventas_brutas_por_pais,
                promedio_ventas.name: promedio_ventas,
                ganancias_totales_por_pais.name: ganancias_totales_por_pais,
                ingresos_totales_por_pais.name: ingresos_totales_por_pais,
                promedio_unidades_vendidas.name: promedio_unidades_vendidas,
                margen_ganancia_medio_por_pais.name: margen_ganancia_medio_por_pais,
                total_ganancias.name : total_ganancias,
                crecimiento_anual_por_pais.name : crecimiento_anual_por_pais
                }
sheet.append(paises)
for nombre, stat in estadisticas.items():
    lista_append = [nombre]
    for dato in stat:
        lista_append.append(round(dato,2))
    sheet.append(lista_append)

#Ajustamos la longitud de los datos
for col in sheet.columns:
    max_length = 0  # Inicializamos la longitud máxima
    col_letter = col[0].column_letter  # Obtener la letra de la columna
    
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))  # Encontrar el texto más largo

    sheet.column_dimensions[col_letter].width = max_length + 2  # Ajustar ancho con un margen extra

#Aplicamos formateado diferencial
for row in sheet.iter_rows(min_row= 2, min_col = 2):
    valores = [int(cell.value) for cell in row]
    minimo = min(valores)
    maximo = max(valores)
    media = sum(valores)/len(valores) 
    color_scale = ColorScaleRule(start_type= "num", start_value = minimo, start_color= "f80a0a",
                             mid_type= "num", mid_value = media, mid_color= "fcf80d",
                             end_type= "num", end_value = maximo, end_color= "7af80a")
    sheet.conditional_formatting.add(f"{row[0].coordinate}:{row[-1].coordinate}", color_scale)

#Añadimos bordes
border_style = Border(left = Side(style= "thin"), right= Side(style = "thin"),
                top = Side(style = "thin"), bottom = Side(style = "thin"))

for row in sheet.iter_rows(min_row=1, max_row=8, min_col=1, max_col=6):
    for cell in row:
        cell.border = border_style

### Creamos tablas a partir de los datos

In [994]:
#Creamos la hoja donde se guardaran las tablas
data.create_sheet("Graficos")
data.active = data["Graficos"]
sheet = data.active

#### Chartbar

In [995]:
from openpyxl.chart import BarChart, PieChart, LineChart, Reference, ScatterChart
paises = Reference(data["Resumenes"], min_col = 2, min_row = 1, max_row = 1, max_col = 6)
#Chartbar para comparar los ingresos brutos y las ganancias de cada pais
bar_chart = BarChart()
gross_data = Reference(data["Resumenes"], min_col=1, min_row=2, max_row=2, max_col = 6)
profit_data = Reference(data["Resumenes"], min_col=1, min_row=4, max_row= 4, max_col = 6)
bar_chart.add_data(gross_data, from_rows= True, titles_from_data= True)
bar_chart.add_data(profit_data,from_rows= True, titles_from_data= True)
bar_chart.set_categories(paises)
bar_chart.title = "Comparación de Ventas y Ganancias"
bar_chart.y_axis.title = "€"
bar_chart.x_axis.title = "Países"
bar_chart.legend.position = "r"  # Posición de la leyenda
bar_chart.style = 18
sheet.add_chart(bar_chart, "C3")

#### Charbar de promedio de ventas

In [997]:
bar_chart = BarChart()
promedio_ventas_data = Reference(data["Resumenes"], min_col=1, min_row=3, max_row=3, max_col = 6)
bar_chart.add_data(promedio_ventas_data, from_rows= True, titles_from_data= True)
bar_chart.set_categories(paises)
bar_chart.title = "Promedio de ventas por país"
bar_chart.y_axis.title = "€"
bar_chart.x_axis.title = "Países"
bar_chart.legend.position = "r"  # Posición de la leyenda
bar_chart.style = 18

sheet.add_chart(bar_chart, "C20")

### PieChart de margen de crecimiento anual

In [998]:
pie_chart = PieChart()
data_crecimiento = Reference(data["Resumenes"], min_col=1, min_row=8, max_row=8, max_col=6)
pie_chart.add_data(data_crecimiento, from_rows= True, titles_from_data= True)
pie_chart.set_categories(paises)
pie_chart.title = "Crecimiento anual por país"
pie_chart.legend_position = "r"
sheet.add_chart(pie_chart, "M20")

In [999]:
data.save("data/Financial Sample.xlsx")